In [9]:
from bs4 import BeautifulSoup
import glob
import pypandoc
from markdownify import markdownify as md
import os


In [111]:
%load_ext dotenv
%dotenv

In [10]:


glob.glob("../ethical/mle/*")

newsletter_files = glob.glob("../ethical/mle/*")

In [281]:

def fetch_clean_newsletter(file):

    f = open(file)
    lines = f.readlines()
    
    html = BeautifulSoup("\n".join(lines), "html.parser")
    main_content = html.find("table").parent


    for tag in main_content():
            for attribute in ["class", "id", "name", "style", "data-darkreader-inline-color", "align"]:
                    del tag[attribute] 
    
    invalid_tags = ["table", "tbody", "thead", "tr", "td", "th", "tfoot", "b", "div", "img", "span", "strong"]
    for tag in invalid_tags:
            for match in main_content.findAll(tag):
                    match.replaceWithChildren()

    table_str = str(main_content)

    for t in invalid_tags:
        table_str = table_str.replace(f"</{t}>", "")
    
    table_str = table_str.replace("<a href=\"#\"></a>", "")
    table_str = table_str.replace("<a href=\'#\'></a>", "")

    table_md_articles = md(table_str)

    table_md_articles = table_md_articles.split("\nUpcoming MLOps Events\n")[0]
    table_md_articles = table_md_articles.split("MLOps = ")[0]
    table_md_articles = table_md_articles.split("MLJobs = ")[0]
    table_md_articles = table_md_articles.split(" OSS: ")[0]

    html_table = pypandoc.convert_text(table_md_articles, "html", format="md", extra_args=('--wrap=none',))
    
    return html_table

In [283]:
newsletters = []

for i in range(len(newsletter_files)):
    newsletter_file = f"../ethical/mle/{i+1}.html"
    print(newsletter_file)
    newsletter = fetch_clean_newsletter(newsletter_file)
    newsletters.append(newsletter)


../ethical/mle/1.html
../ethical/mle/2.html
../ethical/mle/3.html
../ethical/mle/4.html
../ethical/mle/5.html
../ethical/mle/6.html
../ethical/mle/7.html
../ethical/mle/8.html
../ethical/mle/9.html
../ethical/mle/10.html
../ethical/mle/11.html
../ethical/mle/12.html
../ethical/mle/13.html
../ethical/mle/14.html
../ethical/mle/15.html
../ethical/mle/16.html
../ethical/mle/17.html
../ethical/mle/18.html
../ethical/mle/19.html
../ethical/mle/20.html
../ethical/mle/21.html
../ethical/mle/22.html
../ethical/mle/23.html
../ethical/mle/24.html
../ethical/mle/25.html
../ethical/mle/26.html
../ethical/mle/27.html
../ethical/mle/28.html
../ethical/mle/29.html
../ethical/mle/30.html
../ethical/mle/31.html
../ethical/mle/32.html
../ethical/mle/33.html
../ethical/mle/34.html
../ethical/mle/35.html
../ethical/mle/36.html
../ethical/mle/37.html
../ethical/mle/38.html
../ethical/mle/39.html
../ethical/mle/40.html
../ethical/mle/41.html
../ethical/mle/42.html
../ethical/mle/43.html
../ethical/mle/44.ht

In [123]:
import re

In [407]:
newsletter_objs = []

skip_first_ul = [99, 100, 199, 200]
skip = [238]

for i, newsletter in enumerate(newsletters):
    # TODO: 0-23 format
    # TODO: Fix 238 format
    if i <= 22 or i == 238: continue
    
    newsletter_obj = {}
    
    html = BeautifulSoup(newsletter, "html.parser")
    
    #print(re.findall("This week.*:(.*)more[ 🚀]?", html.text, re.S)[0])
    
    uls = html.find_all("ul")
    
    ul = uls[0] if i not in skip_first_ul else uls[1]
    
    newsletter_obj["issue"] = re.findall("Issue\\s+#(\\d+)", html.text)[0]
    
    newsletter_items = []
    for li in ul.find_all("li"):
        newsletter_item = {}
        
        li_link = li.find("a")
        li_text = li.text
        if not li_link: continue
        li_href = li_link["href"]
        
        if "#mljobs" in li_href: continue
        if "AI conferences" in li_text: continue
        if "Open Source ML Frame" in li_text: continue
        if "Awesome AI Guidelines" in li_text: continue
    
        newsletter_item["title"] = li_text
        newsletter_item["link"] = li_href
    
        content_found = html.find_all("a", attrs={"href": li_href})    
        if content_found:
            content_selected = content_found[0]
            if len(content_found) < 2:
                content_found_extra = html.find_all("a", attrs={"href": li_href.replace("http://", "https://")})
                if content_found_extra:
                    content_selected = content_found_extra[0]
            else:
                content_selected = content_found[1]
            # Newsletters before do not have separate links
            if i <= 262:
                content = content_selected.parent.text.strip()
            else:
                content = content_selected.parent.findNext("p").text.strip()
            newsletter_item["content"] = content
        else:
            newsletter_item["content"] = ""
            
        newsletter_items.append(newsletter_item)
    
    newsletter_obj["items"] = newsletter_items

    newsletter_objs.append(newsletter_obj)


In [394]:
html = BeautifulSoup(newsletters[263], "html.parser")
print(html)

<p>Issue #264 🤖</p>
<p>🥳🥳🥳 Happy New Year 2024!! 🥳🥳🥳</p>
<p>This week we CONTINUE to celebrate 5 years since starting this <a href="https://www.linkedin.com/feed/update/urn:li:activity:7146802107847757824/">weekly Machine Learning newsletter</a> 🎉 What started with <a href="https://github.com/EthicalML/ethical/commit/aaedb7a9abed888327abb8a5b158c83accdc7d65?diff=unified&amp;w=0">just one commit</a>, today now has almost 60,000 subscribers 🚀 And not a single Sunday missed 🤯 Thank you to everyone for your continued support - in today’s newsletter we share a special edition celebrating our achievements throughout 2023!</p>
<p>­ ­</p>
<p>Thank you for being part of over <a href="https://ethical.institute/mle.html">50,000+ ML professionals and enthusiasts</a> who receive weekly articles &amp; tutorials on Machine Learning &amp; MLOps 🤖 You can join the newsletter <a href="https://ethical.institute/mle.html">for free at https://ethical.institute/mle.html</a> ⭐</p>
<p>If you like the content 

In [13]:
from langchain.schema.document import Document
from langchain.text_splitter import CharacterTextSplitter

In [48]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

chunks = []

for newsletter in newsletters[:10]:
    texts = text_splitter.split_text(newsletter)
    docs = [Document(page_content=t) for t in texts]
    chunks += text_splitter.create_documents(newsletters)


In [50]:
len(chunks)

3140

In [51]:
for i, chunk in enumerate(chunks):
    chunk.metadata["doc_type"] = f"{i+1}.html"
    chunk.metadata["type"] = f"{i+1}.html"

In [52]:
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma

db_name = "vector_db"


In [54]:

# Delete if already exists
if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()


In [55]:

embeddings = OllamaEmbeddings(model="nomic-embed-text")

# Create vectorstore
vectorstore = None
for i, chunk in enumerate(chunks):
    print(f"Processing {i} of {len(chunks)}")
    if not vectorstore:
        vectorstore = Chroma.from_documents(documents=[chunk], embedding=embeddings, persist_directory=db_name)
    else:
        vectorstore.from_documents(documents=[chunk], embedding=embeddings, persist_directory=db_name)
        
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Processing 0 of 3140
Processing 1 of 3140
Processing 2 of 3140
Processing 3 of 3140
Processing 4 of 3140
Processing 5 of 3140
Processing 6 of 3140
Processing 7 of 3140
Processing 8 of 3140
Processing 9 of 3140
Processing 10 of 3140
Processing 11 of 3140
Processing 12 of 3140
Processing 13 of 3140
Processing 14 of 3140
Processing 15 of 3140
Processing 16 of 3140
Processing 17 of 3140
Processing 18 of 3140
Processing 19 of 3140
Processing 20 of 3140
Processing 21 of 3140
Processing 22 of 3140
Processing 23 of 3140
Processing 24 of 3140
Processing 25 of 3140
Processing 26 of 3140
Processing 27 of 3140
Processing 28 of 3140
Processing 29 of 3140
Processing 30 of 3140
Processing 31 of 3140
Processing 32 of 3140
Processing 33 of 3140
Processing 34 of 3140
Processing 35 of 3140
Processing 36 of 3140
Processing 37 of 3140
Processing 38 of 3140
Processing 39 of 3140
Processing 40 of 3140
Processing 41 of 3140
Processing 42 of 3140
Processing 43 of 3140
Processing 44 of 3140
Processing 45 of 314

In [86]:

db_name2 = "vector_db_2"

embeddings = OllamaEmbeddings(model="llama3.2")

# Delete if already exists
if os.path.exists(db_name2):
    Chroma(persist_directory=db_name2, embedding_function=embeddings).delete_collection()


# Create vectorstore
vectorstore2 = None
for i, newsletter in enumerate(newsletters):
    print(f"Processing {i} of {len(newsletters)}")
    if not vectorstore2:
        vectorstore2 = Chroma.from_texts(texts=[newsletter], embedding=embeddings, persist_directory=db_name2)
    else:
        vectorstore2.from_texts(texts=[newsletter], embedding=embeddings, persist_directory=db_name2)
        
print(f"Vectorstore created with {vectorstore2._collection.count()} documents")

Processing 0 of 314
Processing 1 of 314
Processing 2 of 314
Processing 3 of 314
Processing 4 of 314
Processing 5 of 314
Processing 6 of 314
Processing 7 of 314
Processing 8 of 314
Processing 9 of 314
Processing 10 of 314
Processing 11 of 314
Processing 12 of 314
Processing 13 of 314
Processing 14 of 314
Processing 15 of 314
Processing 16 of 314
Processing 17 of 314
Processing 18 of 314
Processing 19 of 314
Processing 20 of 314
Processing 21 of 314
Processing 22 of 314
Processing 23 of 314
Processing 24 of 314
Processing 25 of 314
Processing 26 of 314
Processing 27 of 314
Processing 28 of 314
Processing 29 of 314
Processing 30 of 314
Processing 31 of 314
Processing 32 of 314
Processing 33 of 314
Processing 34 of 314
Processing 35 of 314
Processing 36 of 314
Processing 37 of 314
Processing 38 of 314
Processing 39 of 314
Processing 40 of 314
Processing 41 of 314
Processing 42 of 314
Processing 43 of 314
Processing 44 of 314
Processing 45 of 314
Processing 46 of 314
Processing 47 of 314
Pr

In [92]:
vectorstore2.similarity_search("data visualisation libraries")

[Document(metadata={}, page_content='<p>THE ML ENGINEER\xa0🤖 Issue #52</p>\n<p>\xa0 \xa0 This week in <a\nhref="https://ethical.institute/mle/52.html">Issue #52:</a></p>\n<ul>\n<li>Top Python <a\nhref="http://tryolabs.com/blog/2019/12/10/top-10-python-libraries-of-2019/">ML\nLibraries in 2019</a></li>\n<li>NeurIPS <a href="http://slideslive.com/neurips/">2019 Videos are\nOut</a></li>\n<li>Modern NLP with <a href="http://changelog.com/practicalai/68">SpaCy\nPodcast</a></li>\n<li>Testing Guide <a href="http://martinfowler.com/testing/">for\nSoftware</a></li>\n<li>Spotify on Better <a\nhref="http://labs.spotify.com/2019/12/13/the-winding-road-to-better-machine-learning-infrastructure-through-tensorflow-extended-and-kubeflow/">ML\nInfrastructure</a></li>\n<li>Featured OSS <a\nhref="http://github.com/ethicalml/awesome-production-machine-learning">Production\nML Libraries</a></li>\n<li>Awesome AI Guidelines <a\nhref="https://github.com/ethicalml/awesome-artificial-intelligence-guidelines">to

In [94]:
collection = vectorstore2._collection
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"The vectors have {dimensions:,} dimensions")

The vectors have 3,072 dimensions


In [95]:
from langchain_ollama import ChatOllama

In [96]:
llm = ChatOllama(
    model = "llama3.2",
    temperature = 0.8,
    num_predict = 256)

In [97]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [105]:

# set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
# ConversationSummaryBufferMemory(llm=llm, input_key='question', output_key='answer')

# the retriever is an abstraction over the VectorStore that will be used during RAG
retriever = vectorstore2.as_retriever()

# putting it together: set up the conversation chain with the GPT 4o-mini LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory, verbose=True)

In [106]:
query = "What are some popular papers from the ML Engineer newsletter?"
result = conversation_chain.invoke({"question":query})
print(result["answer"])



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
<p>THE ML ENGINEER 🤖 Issue #32</p>
<p>  The ML Engineer 🤖 has reached <a
href="https://ethical.institute/mle.html">1400+ subscribers</a> 🚀 and
the open source <a
href="https://github.com/ethicalml/awesome-production-machine-learning">Production
Machine Learning repo</a> has surpassed 1300 stars (over 30% increase in
a week 🔥🔥🔥) a massive thank you to all our subscribers and community
members for all your support ✨👏🎉😃</p>
<p>    This week in <a
href="https://ethical.institute/mle/32.html">Issue #32:</a></p>
<ul>
<li>End-to-end ML <a
href="http://www.oreilly.com/ideas/enabling-end-to-end-machine-learning-pipelines-in-real-world-applications">Pipelines
in Enterprise</a></li>
<li>Code-free deep learning 

In [107]:
def chat(message, history):
    result = conversation_chain.invoke({"question": message})
    return result["answer"]

In [108]:
import gradio as gr

In [109]:
view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.




> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What are some popular papers from the ML Engineer newsletter?
Assistant: The ML Engineer newsletter is a popular resource for machine learning professionals, and it often features articles on the latest research and trends in the field. Here are some popular papers that have been mentioned or featured in the ML Engineer newsletter:

1. "Attention Is All You Need" by Vaswani et al. (2017) - This paper introduced the Transformer architecture, which has become a dominant force in natural language processing and other NLP tasks.

2. "Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift" by Ioffe and Ba (2015) - This paper introduced batch normalization, a technique that has become widely used in deep learning to stabilize tra